# Task Description
we will employ the powerful Genetic Algorithm (GA) to match an experimental measurement (angle distributions or kinetic rates) by optimizing the parameters [h, f, g, d, c, b, a] in the pairwise interaction potentials. Previously, you explored the process of manually adjusting these parameters to gain physical insights on how these parameters will change the angle distributions or kinetic rates. Now, we take a step further and use optimization techniques to automate this process and find the optimal parameter values automatically.

# What is Optimization?
Optimization techniques like Genetic Algorithms are essential when dealing with complex systems and high-dimensional parameter spaces. Manual tuning (trial or error) can be time-consuming, and may not guarantee finding the best set of parameters. With Genetic Algorithm, we can efficiently explore the parameter space, intelligently evolve candidate solutions over generations, and converge towards the best parameter values that can produce desired angle distributions (this desired value is also called **objective functions**).

# What is Genetic Algorithm?
Genetic Algorithms (GAs) are powerful optimization techniques inspired by natural selection and evolution. They solve complex problems by representing candidate solutions (desired parameters) as chromosomes and iteratively improving them. Basically it includes:


**Initialization**: A population of candidate solutions (chromosomes) is randomly generated. Each solution represents a possible solution to the problem at hand.

**Evaluation** (Fitness Function): Each candidate solution is evaluated using a fitness function that quantifies how well it solves the problem. The fitness function assigns a fitness score to each solution, with higher scores indicating better solutions.

**Selection**: Based on their fitness scores, candidate solutions are selected to become parents for the next generation. Solutions with higher fitness have a higher chance of being selected, similar to the survival of the fittest in nature.

**Crossover**: Selected parent solutions undergo crossover or recombination, where portions of their genetic information are exchanged or combined to create new offspring solutions.

**Mutation**: To introduce diversity into the population and avoid getting stuck in local optima, some of the offspring solutions undergo mutation, where small random changes are applied to their genetic information.

# Resources
Please watch the following videos to understand how optimization works (genetic algorithm) in general

1. https://www.youtube.com/watch?v=qiKW1qX97qA&t=129s&ab_channel=LeiosLabs
2. https://www.youtube.com/watch?v=uQj5UNhCPuo&ab_channel=KieCodes (from 4:17)

## Remarks
Feel free to try something more spicy (CMA-ES, Bayesian optimization, or Nelder Mead) if GA sounds too **simple**

#Preset Functions
Don't change anything in this particular section

In [ ]:
"""
Created on Mon May 15 17:19:58 2023

@author: samtari
"""
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit

def processingDump(folder_path):
    f = open('{}/2D-Hinge.lammpstrj'.format(folder_path), 'r')
    fw = open('{}/binding and dissociation.txt'.format(folder_path),'w+')
    data = f.readlines()
    timestep = 0.1 # ps match the timestep in the simulation
    stepStartLineIndexList = []
    stepList = []
    numAtom = 0
    lineIndex = 0
    for line in data:
        if 'ITEM: TIMESTEP' in line:
            stepStartLineIndexList.append(int(lineIndex))
            stepList.append(int(data[lineIndex+1]))
            numAtom = int(data[lineIndex+3])
        lineIndex += 1
    f.close()
    ReactonList = []
    closeFraction = 0
    closeCount = 0
    count = 0
    TimeList = []
    # print(len(stepStartLineIndexList))
    fw.write('Time_ps' + ' ' + 'Status' +'\n')
    for stepStartLineIndex in stepStartLineIndexList[:]:
        if count == 100 or count == 0: # output every this many steps
            count = 0
            step = stepList[stepStartLineIndexList.index(stepStartLineIndex)]
            TimeList.append(step*timestep/1000) # ps -> ns
            print(step)
            for line in data[stepStartLineIndex+9:stepStartLineIndex+9+numAtom]:
                lineSplit = line.split()
                atomID = int(lineSplit[0])
                if atomID == 18:
                    atomType = int(lineSplit[1])
                    if atomType == 7: # hinge open, FRET signal off
                        ReactonList.append(0)
                        fw.write(str(step*timestep) + ' ' + str(0)+'\n')
                    elif atomType == 8: # hinge closed, FRET signal on
                        ReactonList.append(1)
                        closeCount += 1
                        fw.write(str(step*timestep) + ' ' + str(1)+'\n')
        count += 1
    closeFraction = closeCount/len(ReactonList)
    print(len(ReactonList))
    print(closeFraction)
    # Create a figure and axes
    fig, ax = plt.subplots()
    # Customize the y-axis ticks
    yticks = np.arange(0, 1.1, 1)  # Specify the desired tick locations
    ax.set_yticks(yticks)
    ax.plot(TimeList[:10000],ReactonList[:10000]) # only plot part of it
    plt.xlabel('Time (ns)', fontsize=14)
    plt.ylabel('Hinge Status (1=bound, 0=unbound)', fontsize=14)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.tight_layout()
    plt.subplots_adjust(top=0.88)
    plt.title('Hinge Status vs Time (part)', fontsize=14)
    plt.savefig('Hinge Status vs Time.png', dpi=300)
    plt.show()
    fw.close()


    # transition time of binding and dissociation
    f = open('{}/binding and dissociation.txt'.format(folder_path),'r')
    fw = open('{}/transition_time_to_binding.txt'.format(folder_path),'w+')
    fw2 = open('{}/transition_time_to_dissociation.txt'.format(folder_path),'w+')
    lastStatus = 2
    lastTime = 0
    onList = []
    offList = []
    data = f.readlines()
    for line in data[1:]:
        lineSplit = line.split()
        time = float(lineSplit[0])
        status = int(lineSplit[1]) #  hinge closed (1), FRET signal on or hinge open (0), FRET signal off
        if status != lastStatus:
            lastStatus = status
            timeDifference = time - lastTime
            lastTime = time
            # print(timeDifference)
            if status == 1:
                # print(timeDifference, ' 1')
                onList.append(timeDifference)
                fw.write(str(timeDifference)+'\n')
            elif status == 0:
                # print(timeDifference, ' 0')
                offList.append(timeDifference)
                fw2.write(str(timeDifference)+'\n')
    fw.close()
    fw.close()
    f.close()


# probability distribution and binding/dissociation rates
def getTimeAndCount(numberOfBins,fileName):
    f = open(fileName,'r')
    data = f.readlines()
    timeList = []
    # print(data)
    for line in data:
        time = float(line.split()[0])
        timeList.append(time)
    maxTime = max(timeList)
    minTime = min(timeList)
    interval = (maxTime - minTime)/numberOfBins
    timeChunkList = []
    TimeCountList = []
    timeChunkListStr = []
    iList = []
    i = 0
    for chunk in range(numberOfBins):
        i += 1
        iList.append(i)
        currentChunkTimeList = []
        count = 0
        # print(chunk)
        lower = chunk * interval
        upper = (chunk + 1) * interval
        # print(lower,upper)
        for time in timeList:
            if time <= upper and time >lower:
                currentChunkTimeList.append(time)
                count += 1
        TimeCountList.append(count)
        timeChunkList.append(upper)
        timeChunkListStr.append(str(upper/1000)) # ps -> ns
    CountSum = sum(TimeCountList)
    cumulativeCount = 0
    cumulativeCountList = []
    cumulativeProbabilityList = []
    for time in TimeCountList:
        cumulativeCount += time
        cumulativeCountList.append(cumulativeCount)
        cumulativeProbabilityList.append(cumulativeCount/CountSum)
    # plt.plot(timeChunkList,cumulativeProbabilityList)
    # plt.show()
    f.close()
    return interval, timeChunkListStr, timeChunkList, TimeCountList, cumulativeProbabilityList




def plotCountDistributionBar(transition, categories, values):
    # Create a figure and axis
    fig, ax = plt.subplots()
    # Plot the bar chart
    ax.bar(categories, values)
    # Add labels and title
    ax.set_xlabel('Time to Transition (ns)', fontsize=14)
    ax.set_ylabel('Count (-)', fontsize=14)
    ax.tick_params(axis='x', labelsize=14)
    ax.tick_params(axis='y', labelsize=14)
    plt.xticks(rotation=45)
    fig.set_size_inches(8, 6)
    plt.tight_layout()
    plt.subplots_adjust(top=0.88)
    ax.set_title('Distribution of Transition Time to ' + str(transition), fontsize=14)
    plt.savefig('Distribution of Transition Time to ' + str(transition) +'.png', dpi=300)
    plt.show()

# Define the exponential function
def exponential_func(x, a, b, c):
    return -a * np.exp(-b * x) + c


def fitting(timeChunkList,cumulativeProbabilityList):
    # Generate some sample data
    TimeNsList = [i/1000 for i in timeChunkList] # convert time to ns
    x_data = np.array(TimeNsList)
    y_data = np.array(cumulativeProbabilityList)
    # Perform the least squares fit
    params, _ = curve_fit(exponential_func, x_data, y_data)
    # Extract the optimized parameters
    a_fit, b_fit, c_fit = params
    print(a_fit, b_fit, c_fit)
    # Generate points on the fitted exponential curve
    x_fit = np.array(TimeNsList)
    y_fit = exponential_func(x_fit, a_fit, b_fit, c_fit)
    return b_fit, x_data, y_data, x_fit, y_fit

#processingDump()

#interval, timeChunkListStr, timeChunkList, TimeCountList, cumulativeProbabilityList = getTimeAndCount(100, 'transition_time_to_binding.txt')
#__, categories, __, values, __ = getTimeAndCount(10, 'transition_time_to_binding.txt')
#plotCountDistributionBar('Binding',categories, values)

#interval1, timeChunkListStr1, timeChunkList1, TimeCountList1, cumulativeProbabilityList1 = getTimeAndCount(100, 'transition_time_to_dissociation.txt')
#__, categories1, __, values1, __ = getTimeAndCount(10, 'transition_time_to_dissociation.txt')
#plotCountDistributionBar('Dissociation',categories1, values1)

#b_fit, x_data, y_data, x_fit, y_fit = fitting(timeChunkList,cumulativeProbabilityList)
#b_fit1, x_data1, y_data1, x_fit1, y_fit1 = fitting(timeChunkList1,cumulativeProbabilityList1)

# Plot the original data and the fitted curve
#plt.scatter(x_data, y_data, color='black', marker='s', facecolors='none', label='Binding')
#plt.scatter(x_data1, y_data1, color='black', marker='o', facecolors='none',label='Dissociation')
#plt.plot(x_fit, y_fit,'r-', label='Fit (Binding Rate = '+str(round(b_fit, 3))+' ns$^{-1}$)')
#plt.plot(x_fit1, y_fit1,'b-', label='Fit (Dissociation Rate = '+str(round(b_fit1, 3))+' ns$^{-1}$)')
#plt.xticks(fontsize=14)
#plt.yticks(fontsize=14)
#plt.xlabel('Time to Transition (ns)', fontsize=14)
#plt.ylabel('Cumulative Probability', fontsize=14)
#plt.legend(loc='best')
#plt.tight_layout()
#plt.subplots_adjust(top=0.88)
#plt.title('Cumulative Probability', fontsize=14)
#plt.savefig('Cumulative Probability and Exponential Fitting.png', dpi=300)
#plt.show()


# Functions
Work on the code section labeled with "TODO".
Don't worry too much about the other functions listed here. Once you finish this stage 1 jupyterbook, we will start implementing optimization algorithms (part of it) and these functions will be useful during that stage

In [ ]:
import os
import shutil
import tabulate
import math
files_list = ['Reaction.Map', 'Hinge.in','PairPreReacted.molecule','PairPostReacted.molecule','2DHinge.data'] #a list of file you should have in the same folder as this jupyter notebook before you run this code

def insert_folder(start, end):
    data =[]
    for x in range(start, end +1):
        a = 2.5
        b = 100
        c = 0.1
        d = 6
        h = 14.8
        g = 0.2
        f = 0

        calculated_form = (1 / (x ** a)) -b * math.exp(-c * ((x-d) ** 2)) + 2 * math.exp(-g * ((x-h) **2))
        calculated_val = calculated_form/x
        data.append([x, calculated_form, calculated_val])
        table = tabulate(data)
        return
def Write_parameters(folder_path, parameter_set):
  '''
  This function takes an array parameter_set [h,f,g,d,c,b,a] and generate bond table file for LAMMPS to read.
  TODO: Follow your previous approach to generate such file in python, given the parameter set h,f,g,d,c,b,a
  '''
  o = open('{}/Bond.table',format(folder_path), 'w')
  o.write("ENTRY1\n")
  o.write("N 870\n")
  # your function to output distance, energy, and force
  data = insert_folder(1,869)
  for line in data:

    o.write("{} {} {} {}".format(line[0],line[0],line[1],line[2]))
    o.write("\n")

  o.close()
  return
def copy(src, dst):
  '''
  This function copies files from scr to dst
  '''
  if os.path.isdir(dst):
      dst = os.path.join(dst, os.path.basename(src))
  shutil.copyfile(src, dst)

def Prepare_files_in_folder(folder_path, parameter_set):
  '''
  This function prepares a subfolder containing simulation files with name containing generation number and identifier.
  '''
  !mkdir {folder_path}
  for f in files_list:
      copy(f,folder_path)
  Write_parameters(folder_path, parameter_set)
  return folder_path

def Run_simulations(folder_path, parameter_set):
  !lmp_serial -in {folder_path}/Hinge.in > {folder_path}/output.out
  return

def Read_simulations_results(folder_path):
  # replace TODO with 'transition_time_to_dissociation.txt'if you are finding dissociation rate, or 'transition_time_to_assoiciation.txt' if you are finding association rate
  interval, timeChunkListStr, timeChunkList, TimeCountList, cumulativeProbabilityList = getTimeAndCount(100, '{}/transition_time_to_assoiciation.txt'.format(folder_path))
  b_fit, x_data, y_data, x_fit, y_fit = fitting(timeChunkList,cumulativeProbabilityList)
  return b_fit

def Function_to_be_optimized(identifier, parameter_set):
  folder_path = '{}'.format(identifier)
  Prepare_files_in_folder(folder_path, parameter_set)
  Run_simulations(folder_path, parameter_set)
  processingDump(folder_path)
  b_fit = Read_simulations_results(folder_path)
  return b_fit

In [ ]:
import random

# Genetic Algorithm Parameters
POPULATION_SIZE = 100
GENOME_LENGTH = 7
MUTATION_RATE = 0.01
TOURNAMENT_SIZE = 3
GENERATIONS = 100

# Define the range for each parameter
#What is a reasonable range of these value?
PARAMETER_RANGES = {
    'h': (0, 76),
    'f': (1, 3.5),
    'g': (-5, 18),
    'd': (0, 2),
    'c': (-10, 11),
    'b': (-20, 116),
    'a': (0, 11)
}

# Generate initial population with continuous variables in their specified ranges
def generate_population(pop_size, genome_length):
    population = []
    for _ in range(pop_size):
        chromosome = [random.uniform(PARAMETER_RANGES[param][0], PARAMETER_RANGES[param][1]) for param in sorted(PARAMETER_RANGES)]
        population.append(chromosome)
    return population

# Evaluate fitness of each individual (modify this according to your problem)
def fitness_function(chromosome):
    '''
    TODO
    Given the experimental value = 0.223, what should the objective function be so that when the obtained value is close to this
    experimental value, we get high fintess score? Choose your own objective function, starting with the mean squared error might be a good idea
    '''
    h,f,g,d,c,b,a = chromosome
    identifier = "{:.2f}_{:.2f}_{:.2f}_{:.2f}_{:.2f}_{:.2f}_{:.2f}".format(h,f,g,d,c,b,a) #set an "id" for each simulation, which is the combination of these simulations

    #b_fit = Function_to_be_optimized(identifier, chromosome)
    #score = (0.233-b_fit)**2
    score = sum(chromosome)
    return score

# Tournament selection to choose parents for mating
def tournament_selection(population):
    selected_parents = []
    for _ in range(len(population)):
        participants = random.sample(population, TOURNAMENT_SIZE)
        winner = max(participants, key=fitness_function)
        selected_parents.append(winner)
    return selected_parents

# Crossover two parents to create two children (single-point crossover)
def crossover(parent1, parent2):
    crossover_point = random.randint(1, GENOME_LENGTH - 1)
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2

# Swap mutation for continuous variables
def mutate(individual, mutation_rate):
    for i in range(GENOME_LENGTH):
        if random.random() < mutation_rate:
            min_value, max_value = PARAMETER_RANGES[sorted(PARAMETER_RANGES)[i]]
            individual[i] = random.uniform(min_value, max_value)

# Genetic Algorithm main function
def genetic_algorithm():
    # Generate initial population
    population = generate_population(POPULATION_SIZE, GENOME_LENGTH)

    for generation in range(GENERATIONS):

        # Tournament selection
        selected_parents = tournament_selection(population)

        # Create new generation
        new_generation = []
        while len(new_generation) < POPULATION_SIZE:
            parent1, parent2 = random.sample(selected_parents, 2)
            child1, child2 = crossover(parent1, parent2)
            mutate(child1, MUTATION_RATE)
            mutate(child2, MUTATION_RATE)
            new_generation.extend([child1, child2])

        # Replace the old population with the new one
        population = new_generation

        # Display the best individual in each generation (optional)
        best_individual = max(population, key=fitness_function)
        print(f"Generation {generation}: Best Individual: {best_individual}, Fitness: {fitness_function(best_individual)}")

    # Final result
    best_individual = max(population, key=fitness_function)
    print("\nFinal Result:")
    print(f"Best Individual: {best_individual}")
    print(f"Fitness: {fitness_function(best_individual)}")

In [ ]:
genetic_algorithm()

Generation 0: Best Individual: [8.179881268149952, 109.14054012327708, 8.760472733780421, 0.5491227046421132, 2.8014089604926515, 15.639347462730782, 60.43996382224325], Fitness: 205.51073707531626
Generation 1: Best Individual: [10.343590781940494, 108.77532333735468, 6.84976591660222, 0.16834930481777377, 1.8931481959239331, 12.550885143885715, 73.02242988367176], Fitness: 213.60349256419659
Generation 2: Best Individual: [10.343590781940494, 108.77532333735468, 6.84976591660222, 0.8640315499111606, 3.0736697504072175, 12.550885143885715, 73.02242988367176], Fitness: 215.47969636377326
Generation 3: Best Individual: [10.343590781940494, 108.77532333735468, 6.84976591660222, 0.16834930481777377, 2.2739933166775534, 17.506425078431704, 69.29229883642243], Fitness: 215.20974657224684
Generation 4: Best Individual: [10.343590781940494, 108.77532333735468, 6.84976591660222, 1.3784211913323643, 2.5053102427691796, 12.550885143885715, 73.02242988367176], Fitness: 215.4257264975564
Generatio